In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce
import copy

In [2]:
merged_data = pd.read_csv('trained_datas.csv',index_col=0)

In [4]:
merged_data = merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
merged_data

,sex,race,marital-status,occupation,pregnant,relationship,education,income,native-country,workclass,...,kurtosis_glucose,kurtosis_oxygen,mean_glucose,mean_oxygen,skewness_glucose,skewness_oxygen,std_glucose,std_oxygen,age,class
0,Male,White,Never-married,Adm-clerical,f,Not_in_family,Masters,>50K,United-States,State-gov,...,1.280503,3.513019,95.351562,14.103679,3.351277,11.305999,41.828816,47.474805,52.0,1.0
1,Male,Black,Divorced,transport-moving,f,Not_in_family,Some-college,<=50K,United-States,Private,...,0.267679,8.402489,111.164062,2.902174,-0.141940,79.618941,51.802174,19.058253,49.0,0.0
2,Male,White,Never-married,Sales,f,Not_in_family,Some-college,<=50K,United-States,Private,...,5.600391,1.042517,14.460938,57.382107,34.223119,0.231636,29.387737,68.262860,65.0,1.0
3,Male,White,Married-civ-spouse,Craft-repair,f,Husband,Some-college,<=50K,United-States,Private,...,0.510415,10.508914,107.796875,1.910535,1.672603,120.443237,41.562709,17.340950,50.0,1.0
4,Male,White,Married-civ-spouse,Machine-op-inspct,f,Husband,Some-college,<=50K,United-States,Private,...,0.459786,12.595281,111.484375,1.599498,-0.116594,226.699206,53.068350,11.347475,69.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,Male,Asian-Pac-Islander,Married-civ-spouse,transport-moving,f,Husband,HS-grad,<=50K,United-States,Private,...,0.552355,2.398916,103.578125,21.821070,0.777646,4.759217,45.166638,51.177177,67.0,1.0
3929,female,White,Never-married,Prof-specialty,f,Not_in_family,Bachelors,<=50K,United-States,Private,...,4.430580,4.772329,27.468750,7.814381,20.844460,24.500064,37.283110,31.365559,67.0,1.0
3930,female,White,Never-married,Adm-clerical,t,Not_in_family,Some-college,<=50K,United-States,Private,...,0.251239,5.876207,100.859375,4.968227,3.491732,36.871068,32.731324,25.765971,49.0,0.0
3931,Male,White,Married-civ-spouse,transport-moving,f,Husband,7th-8th,<=50K,United-States,Private,...,3.933903,0.558779,40.398438,96.349498,19.294670,-0.717264,33.748458,76.576780,42.0,1.0


In [13]:
categoric = merged_data.select_dtypes(include = ['object'])
print(categoric)

encoder = ce.OneHotEncoder()
encoder.fit(categoric)
transformed = encoder.transform(categoric)
transformed

          sex                 race       marital-status          occupation  \
0        Male                White        Never-married        Adm-clerical   
1        Male                Black             Divorced    transport-moving   
2        Male                White        Never-married               Sales   
3        Male                White   Married-civ-spouse        Craft-repair   
4        Male                White   Married-civ-spouse   Machine-op-inspct   
...       ...                  ...                  ...                 ...   
3928     Male   Asian-Pac-Islander   Married-civ-spouse    transport-moving   
3929   female                White        Never-married      Prof-specialty   
3930   female                White        Never-married        Adm-clerical   
3931     Male                White   Married-civ-spouse    transport-moving   
3932     Male   Asian-Pac-Islander        Never-married        Craft-repair   

     pregnant    relationship      education income

,sex_1,sex_2,race_1,race_2,race_3,race_4,race_5,marital-status_1,marital-status_2,marital-status_3,...,native-country_37,native-country_38,native-country_39,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7
0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3929,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3930,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3931,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [15]:
merged_data = merged_data.drop(categoric, axis = 1)
merged_data

,education-num,capital-gain,fnlwgt,hours-per-week,capital-loss,kurtosis_glucose,kurtosis_oxygen,mean_glucose,mean_oxygen,skewness_glucose,skewness_oxygen,std_glucose,std_oxygen,age,class
0,1.595991,-0.307122,1.389201,2.451872,-0.227166,1.280503,3.513019,95.351562,14.103679,3.351277,11.305999,41.828816,47.474805,52.0,1.0
1,-0.060364,-0.307122,0.215817,1.190632,-0.227166,0.267679,8.402489,111.164062,2.902174,-0.141940,79.618941,51.802174,19.058253,49.0,0.0
2,-0.060364,-0.307122,0.276511,0.774578,-0.227166,5.600391,1.042517,14.460938,57.382107,34.223119,0.231636,29.387737,68.262860,65.0,1.0
3,-0.060364,-0.307122,-0.437769,-2.727388,-0.227166,0.510415,10.508914,107.796875,1.910535,1.672603,120.443237,41.562709,17.340950,50.0,1.0
4,-0.060364,-0.307122,0.055624,2.451872,-0.227166,0.459786,12.595281,111.484375,1.599498,-0.116594,226.699206,53.068350,11.347475,69.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,-0.447286,-0.307122,0.674192,-0.049757,-0.227166,0.552355,2.398916,103.578125,21.821070,0.777646,4.759217,45.166638,51.177177,67.0,1.0
3929,1.166759,-0.307122,-0.156042,1.190632,-0.227166,4.430580,4.772329,27.468750,7.814381,20.844460,24.500064,37.283110,31.365559,67.0,1.0
3930,-0.060364,-0.307122,1.665611,-0.049757,-0.227166,0.251239,5.876207,100.859375,4.968227,3.491732,36.871068,32.731324,25.765971,49.0,0.0
3931,-2.176550,-0.307122,0.274630,0.361031,-0.227166,3.933903,0.558779,40.398438,96.349498,19.294670,-0.717264,33.748458,76.576780,42.0,1.0


In [77]:
merged_data = merged_data.merge(transformed, left_index=True, right_index=True)
merged_data

,education-num,capital-gain,fnlwgt,hours-per-week,capital-loss,kurtosis_glucose,kurtosis_oxygen,mean_glucose,mean_oxygen,skewness_glucose,...,native-country_37,native-country_38,native-country_39,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7
0,1.595991,-0.307122,1.389201,2.451872,-0.227166,1.280503,3.513019,95.351562,14.103679,3.351277,...,0,0,0,1,0,0,0,0,0,0
1,-0.060364,-0.307122,0.215817,1.190632,-0.227166,0.267679,8.402489,111.164062,2.902174,-0.141940,...,0,0,0,0,1,0,0,0,0,0
2,-0.060364,-0.307122,0.276511,0.774578,-0.227166,5.600391,1.042517,14.460938,57.382107,34.223119,...,0,0,0,0,1,0,0,0,0,0
3,-0.060364,-0.307122,-0.437769,-2.727388,-0.227166,0.510415,10.508914,107.796875,1.910535,1.672603,...,0,0,0,0,1,0,0,0,0,0
4,-0.060364,-0.307122,0.055624,2.451872,-0.227166,0.459786,12.595281,111.484375,1.599498,-0.116594,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,-0.447286,-0.307122,0.674192,-0.049757,-0.227166,0.552355,2.398916,103.578125,21.821070,0.777646,...,0,0,0,0,1,0,0,0,0,0
3929,1.166759,-0.307122,-0.156042,1.190632,-0.227166,4.430580,4.772329,27.468750,7.814381,20.844460,...,0,0,0,0,1,0,0,0,0,0
3930,-0.060364,-0.307122,1.665611,-0.049757,-0.227166,0.251239,5.876207,100.859375,4.968227,3.491732,...,0,0,0,0,1,0,0,0,0,0
3931,-2.176550,-0.307122,0.274630,0.361031,-0.227166,3.933903,0.558779,40.398438,96.349498,19.294670,...,0,0,0,0,1,0,0,0,0,0


In [82]:
cls = DecisionTreeClassifier(criterion = 'entropy')

X = merged_data[merged_data.columns.difference(['class'])]
y = merged_data['class']

cls.fit(X, y)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')